In [1]:
import os
import numpy
import sys
import pandas
import rospy
import mujoco_py
from sensor_msgs.msg import JointState
from std_msgs.msg import Float64MultiArray
from std_msgs.msg import String
from geometry_msgs.msg import WrenchStamped
from trajectory_msgs.msg import JointTrajectory, JointTrajectoryPoint
from realsense2_camera.msg import CustomVector
from realsense2_camera.msg import EstimatedPose
from gym.envs.robotics import rotations

In /home/lars/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/lars/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/lars/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/lars/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/lars/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_c

In [2]:
def q_callback(data):
    global qpos
    qpos= numpy.array(data.position)[[2,1,0,3,4,5]]
    
def rad2deg(rad):
    deg= rad*(180/numpy.pi)
    return deg

def traj_control(q):
    dt = 10
    global ctrl_q
    command = JointTrajectory()
    point = JointTrajectoryPoint()
    
    #command.header.stamp = rospy.Time.now()
    #command.header.frame_id = "DRL_UR10"
    #command.joint_names = ["elbow_joint", "shoulder_lift_joint", "shoulder_pan_joint", "wrist_1_joint", "wrist_2_joint", "wrist_3_joint"]
    
    #command.header.stamp = rospy.Time.now()
    command.header.frame_id = "DRL_UR10"
    command.joint_names.append("shoulder_pan_joint")
    command.joint_names.append("shoulder_lift_joint")
    command.joint_names.append("elbow_joint")
    command.joint_names.append("wrist_1_joint")
    command.joint_names.append("wrist_2_joint")
    command.joint_names.append("wrist_3_joint")
    

    point.positions.append(q[0])
    point.positions.append(q[1])
    point.positions.append(q[2])
    point.positions.append(q[3])
    point.positions.append(q[4])
    point.positions.append(q[5])
    
    command.points.append(point)
    command.points[0].time_from_start=rospy.Duration.from_sec(dt)
    
    traj_control_publisher.publish(command)
    ctrl_q = q.copy()

#numpy.savetxt(sys.stdout, qpos, '%5.2f')

In [3]:
rospy.init_node("DRL_Calibration")
rospy.Subscriber("/joint_states", JointState, q_callback)
#traj_control_publisher = rospy.Publisher("/pos_traj_controller/command", JointTrajectory, queue_size=1)

In [4]:
goal_rad = numpy.array([0.00, -1.14, 1.85, -0.70, 1.55, 0.00])
start_rad = numpy.array([1.19467746e-04, -1.30006248e+00,  2.10014057e+00, -8.00064389e-01, 1.57103789e+00, 1.19467746e-04])
goal_deg = [rad2deg(elem) for elem in goal_rad]
start_deg = [rad2deg(elem) for elem in start_rad]

In [5]:
traj_control(goal_rad)

In [5]:
qpos

array([ 4.40832227e-03, -1.13823349e+00,  1.86037254e+00, -7.23192994e-01,
        1.59764516e+00, -1.79592763e-04])

# SIMULATION

In [6]:
HOME_PATH = os.getenv("HOME")
XML_MODEL_PATH = os.path.join(*[HOME_PATH, "DRL_AI4RoMoCo", "code", "environment","UR10_single","ur10_heg.xml"])

In [7]:
def normalize_rad(angles):
    angles = numpy.array(angles)
    angles = angles % (2*numpy.pi)
    angles = (angles + 2*numpy.pi) % (2*numpy.pi)
    for i in range(len(angles)):
        if (angles[i] > numpy.pi):
            angles[i] -= 2*numpy.pi
    return angles

In [8]:
def set_state(qpos):
    #assert qpos.shape == (model.nq,)
    old_state = sim.get_state()
    new_state = mujoco_py.MjSimState(old_state.time, qpos, old_state.qvel,
                                     old_state.act, old_state.udd_state)
    sim.set_state(new_state)
    sim.forward()

In [9]:
model = mujoco_py.load_model_from_path(XML_MODEL_PATH)
sim = mujoco_py.MjSim(model)
q_init = numpy.array([0, -1.3, 2.1, -0.80, 1.5708, 0.0])
q_goal = numpy.array([0.00, -1.14, 1.85, -0.70, 1.55, 0.00])

ctrl_q = q_init
set_state(qpos)

xpos = sim.data.get_body_xpos("gripper_dummy_heg")
x_mat = sim.data.get_body_xmat("gripper_dummy_heg")
rpy = normalize_rad(rotations.mat2euler(x_mat))

#viewer = mujoco_py.MjViewer(sim)
#viewer.render()

In [10]:
xpos

array([ 0.69539974, -0.83325208,  1.17396257])

In [11]:
rpy

array([-1.57092218,  0.02244061,  3.14048138])

In [12]:
goal = numpy.concatenate([xpos,rpy])

In [13]:
goal

array([ 0.69539974, -0.83325208,  1.17396257, -1.57092218,  0.02244061,
        3.14048138])